In [1]:
import os
import re
import sys
from pathlib import Path

import h5py
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc
from sklearn import preprocessing
from tqdm.notebook import tqdm, trange
import anndata as ad
from sklearn.model_selection import train_test_split
import seaborn as sns

In [2]:
d_dir = (Path().cwd().parents[0].parents[0]).absolute()
data_dir = d_dir / "09_datasets"

p_dir = (Path().cwd().parents[0]).absolute()

In [3]:
%load_ext autoreload
%autoreload 2

module_path = str(p_dir / "src")

if module_path not in sys.path:
    sys.path.append(module_path)


In [4]:
import graph
import torch
import torch_geometric.utils
import networkx as nx
import lightning.pytorch as pl
import torch.utils.data as data

p_dir = 'Y:\\coskun-lab\\Thomas\\11_snowflakes\\'
spatial_omics_folder = p_dir + 'data\\' + 'spatial_omics_graph\\'
process_path = p_dir + 'data\\' + 'torch_graph_data\\'
morph_path = p_dir + 'data\\' + 'morph\\' + 'outlinePCA.pkl'

#spatial_omics_folder = (Path().cwd().parents[0]).absolute() / 'data' / 'spatial_omics_graph'
#process_path = (Path().cwd().parents[0]).absolute() / 'data' / 'torch_graph_data'
#morph_path = (Path().cwd().parents[0]).absolute() / 'data' / 'morph' / 'outlinePCA.pkl'

C:\Users\eozturk7\AppData\Local\miniconda3\envs\torch_gpu2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Get  adata

In [29]:
# adata_dir = (Path().cwd().parents[0]).absolute() / 'data'/ 'processed'
# adatas = []
# datasets = []

# # Loop through data
# for (dirpath, dirnames, filenames) in os.walk(adata_dir):
#     for name in tqdm(sorted(filenames)):
#         if 'h5ad' not in name:
#             continue
        
#         # read adata
#         data_name = name.split('.')[0]
#         if '07' not in name:
#             continue
        
#         print(f'Processing Data {data_name}')   
#         path = os.path.join(dirpath, name)
        
#         adata = sc.read_h5ad(path)
#         print(adata.var_names)

# Create data loader

In [6]:
from torch_geometric.loader import DataLoader
seed = torch.Generator().manual_seed(42)

name = 'Breast'

# 0: Tonsil, 1: Adenoid

# Crate dataset
dataset = graph.GraphDatasetMLP(process_path + name, process_path + name + '\\' + 'info.csv', 2, y_name='label')

train_set, val_set, test_set = graph.train_test_val_split(dataset)

# Create Dataloader
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32, shuffle=True)
test_loader = DataLoader(test_set, batch_size=32, shuffle=False)


In [7]:
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.nc}')

Dataset: GraphDatasetMLP(518):
Number of graphs: 518
Number of features: 39
Number of classes: 2


In [8]:
print(f'Train set: {len(train_set)}, val set: {len(test_set)}, val set: {len(val_set)}')

Train set: 249, val set: 207, val set: 62


In [10]:
Xs = []
X_features = []
ys = []
for step, data in enumerate(DataLoader(dataset, batch_size=128)):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    Xs.append(data.x.numpy())
    ys.append(data.y.numpy())
    #X_features.append(data.features.numpy())

Step 1:
Number of graphs in the current batch: 128
DataBatch(x=[128, 39], y=[128], name=[128], batch=[128], ptr=[129])
Step 2:
Number of graphs in the current batch: 128
DataBatch(x=[128, 39], y=[128], name=[128], batch=[128], ptr=[129])
Step 3:
Number of graphs in the current batch: 128
DataBatch(x=[128, 39], y=[128], name=[128], batch=[128], ptr=[129])
Step 4:
Number of graphs in the current batch: 128
DataBatch(x=[128, 39], y=[128], name=[128], batch=[128], ptr=[129])
Step 5:
Number of graphs in the current batch: 6
DataBatch(x=[6, 39], y=[6], name=[6], batch=[6], ptr=[7])


In [14]:
X_intensity = np.concatenate(Xs)
#X_feature = np.concatenate(X_features)


#X = np.concatenate([X_intensity, X_feature], axis=1)
X = X_intensity
y = np.concatenate(ys)

In [15]:
print(X.shape, y.shape)

(518, 39) (518,)


In [16]:
feature_name = data.features_names[0]

AttributeError: 'GlobalStorage' object has no attribute 'features_names'

# Train network K Fold

In [17]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import preprocessing, metrics
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import train_test_split, KFold, ShuffleSplit
from sklearn.utils.class_weight import compute_class_weight
import wandb

In [18]:
from sklearn.model_selection import KFold, StratifiedKFold
from torch.utils.data import SubsetRandomSampler

In [19]:
condition = 'ML_01082024_2'
project_name = f'Breast_{condition}'

In [20]:
models = {
    'Adaboost': AdaBoostClassifier(),
    'DecisionTree': DecisionTreeClassifier(),
    'GradientBoosting' : GradientBoostingClassifier(),
    'NaiveBayes': GaussianNB(),
    'RandomForest': RandomForestClassifier(), 
    'SVM': SVC(probability =True), 
    'MLP': MLPClassifier(random_state=1, max_iter=100, hidden_layer_sizes=[16, 16, 16])
}

In [21]:
X_scaled_ori = preprocessing.MinMaxScaler().fit_transform(X)

In [43]:
class_name = ['Tonsil', 'Adenoid']

## Expression Only

In [44]:
X_scaled = X_scaled_ori[:, :5]
print(X_scaled.shape)

# features = ['CD21', 'Hoeschst', 'CD20', 'CD68', 'Ki67', 'CD138', 'DC-SIGN']
features = ['CD21','CD20', 'CD68', 'Ki67', 'CD138']
print(len(features))

assert X_scaled.shape[1] == len(features)

(442, 5)
5


In [45]:
# # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)
# # kfold = KFold(n_splits = 3, shuffle = True, random_state = 42)
# kfold = ShuffleSplit(test_size=.25, random_state=0)

# for model_name, model in models.items():
#     for k, (train_index, test_index) in enumerate(kfold.split(X)):
#         # Split the dataset
#         X_train, X_test = X_scaled[train_index], X_scaled[test_index]
#         y_train, y_test = y[train_index], y[test_index]
        
#         model.fit(X_train, y_train)
#         y_pred = model.predict(X_test)
#         y_probas = model.predict_proba(X_test)

#         run = wandb.init(project=project_name, group=model_name+'_mean', name=model_name+f'_mean_{k}')
#         wandb.sklearn.plot_classifier(model, 
#                                   X_train, X_test, 
#                                   y_train, y_test, 
#                                   y_pred, y_probas, 
#                                   class_name, 
#                                   is_binary=True, 
#                                   model_name=model_name+'_morph', 
#                                   feature_names=features)
#         wandb.log({'roc': wandb.plots.ROC(y_test, y_probas, class_name)})
#         wandb.log({'pr': wandb.plots.precision_recall(y_test, y_probas, class_name)})

#         accuracy = metrics.accuracy_score(y_test, y_pred)
#         b_accuracy = metrics.balanced_accuracy_score(y_test, y_pred)
#         f1 = metrics.f1_score(y_test, y_pred)
#         auc = metrics.roc_auc_score(y_test, y_probas[:, 1])
#         wandb.log({"accuracy": accuracy, 'b_accuracy': b_accuracy, 'f1':f1, 'auc': auc})
#     run.finish()


## Morphology only 

In [46]:
X_scaled = X_scaled_ori[:, 5:]
print(X_scaled.shape)

features = feature_name
print(len(features))

assert X_scaled.shape[1] == len(features)

(442, 33)
33


In [47]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)
# kfold = KFold(n_splits = 3, shuffle = True, random_state = 0)
kfold = ShuffleSplit(test_size=.25, random_state=0)

for model_name, model in models.items():
    for k, (train_index, test_index) in enumerate(kfold.split(X)):
        # Split the dataset
        X_train, X_test = X_scaled[train_index], X_scaled[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_probas = model.predict_proba(X_test)

        run = wandb.init(project=project_name, group=model_name+'_morph', name=model_name+f'_morph_{k}')
        wandb.sklearn.plot_classifier(model, 
                                  X_train, X_test, 
                                  y_train, y_test, 
                                  y_pred, y_probas, 
                                  class_name, 
                                  is_binary=True, 
                                  model_name=model_name+'_morph', 
                                  feature_names=features)
        wandb.log({'roc': wandb.plots.ROC(y_test, y_probas, class_name)})
        wandb.log({'pr': wandb.plots.precision_recall(y_test, y_probas, class_name)})

        accuracy = metrics.accuracy_score(y_test, y_pred)
        b_accuracy = metrics.balanced_accuracy_score(y_test, y_pred)
        f1 = metrics.f1_score(y_test, y_pred)
        auc = metrics.roc_auc_score(y_test, y_probas[:, 1])
        wandb.log({"accuracy": accuracy, 'b_accuracy': b_accuracy, 'f1':f1, 'auc': auc})
    run.finish()


wandb: 
wandb: Plotting Adaboost_morph.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
wandb: Logged calibration curve.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.65766
auc,0.63724
b_accuracy,0.50309
f1,0.20833


wandb: 
wandb: Plotting Adaboost_morph.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
wandb: Logged calibration curve.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.7027
auc,0.6515
b_accuracy,0.59593
f1,0.4


wandb: 
wandb: Plotting Adaboost_morph.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
wandb: Logged calibration curve.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


...

accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.66667
auc,0.6366
b_accuracy,0.5
f1,0.0


wandb: 
wandb: Plotting MLP_morph.
wandb: WARNING could not find any of attributes feature_importances_, feature_log_prob_, coef_ on classifier. Cannot plot feature importances.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
C:\Users\thu71\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
wandb: Logged summary metrics.
wandb: Logged class proportions.
C:\Users\thu71\AppData\Roaming\Python\Python39\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\thu71\AppData\Roaming\Python\Python39\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: Conver

accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.83784
auc,0.64098
b_accuracy,0.5
f1,0.0


wandb: 
wandb: Plotting MLP_morph.
wandb: WARNING could not find any of attributes feature_importances_, feature_log_prob_, coef_ on classifier. Cannot plot feature importances.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
C:\Users\thu71\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
wandb: Logged summary metrics.
wandb: Logged class proportions.
C:\Users\thu71\AppData\Roaming\Python\Python39\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\thu71\AppData\Roaming\Python\Python39\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: Conver

accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.69369
auc,0.62452
b_accuracy,0.5
f1,0.0


## Both

In [48]:
X_scaled = X_scaled_ori
print(X_scaled.shape)

# features = ['CD21', 'Hoeschst', 'CD20', 'CD68', 'Ki67', 'CD138', 'DC-SIGN'] + feature_name
features =  ['CD21','CD20', 'CD68', 'Ki67', 'CD138'] + feature_name

print(len(features))

assert X_scaled.shape[1] == len(features)



(442, 38)
38


In [49]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)
# kfold = KFold(n_splits = 3, shuffle = True, random_state = 0)
kfold = ShuffleSplit(test_size=.25, random_state=0)

for model_name, model in models.items():
    for k, (train_index, test_index) in enumerate(kfold.split(X)):
        # Split the dataset
        X_train, X_test = X_scaled[train_index], X_scaled[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_probas = model.predict_proba(X_test)

        run = wandb.init(project=project_name, group=model_name+'_morph2', name=model_name+f'_morph2_{k}')
        wandb.sklearn.plot_classifier(model, 
                                  X_train, X_test, 
                                  y_train, y_test, 
                                  y_pred, y_probas, 
                                  class_name, 
                                  is_binary=True, 
                                  model_name=model_name+'_morph2', 
                                  feature_names=features)
        wandb.log({'roc': wandb.plots.ROC(y_test, y_probas, class_name)})
        wandb.log({'pr': wandb.plots.precision_recall(y_test, y_probas, class_name)})

        accuracy = metrics.accuracy_score(y_test, y_pred)
        b_accuracy = metrics.balanced_accuracy_score(y_test, y_pred)
        f1 = metrics.f1_score(y_test, y_pred)
        auc = metrics.roc_auc_score(y_test, y_probas[:, 1])
        wandb.log({"accuracy": accuracy, 'b_accuracy': b_accuracy, 'f1':f1, 'auc': auc})
    run.finish()


wandb: 
wandb: Plotting Adaboost_morph2.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
wandb: Logged calibration curve.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.72973
auc,0.78189
b_accuracy,0.62593
f1,0.44444


wandb: 
wandb: Plotting Adaboost_morph2.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
wandb: Logged calibration curve.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.72973
auc,0.71994
b_accuracy,0.67069
f1,0.53125


wandb: 
wandb: Plotting Adaboost_morph2.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
wandb: Logged calibration curve.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


...

accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.66667
auc,0.66143
b_accuracy,0.5
f1,0.0


wandb: 
wandb: Plotting MLP_morph2.
wandb: WARNING could not find any of attributes feature_importances_, feature_log_prob_, coef_ on classifier. Cannot plot feature importances.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
C:\Users\thu71\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
wandb: Logged summary metrics.
wandb: Logged class proportions.
C:\Users\thu71\AppData\Roaming\Python\Python39\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\thu71\AppData\Roaming\Python\Python39\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: Conve

accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.83784
auc,0.57706
b_accuracy,0.5
f1,0.0


wandb: 
wandb: Plotting MLP_morph2.
wandb: WARNING could not find any of attributes feature_importances_, feature_log_prob_, coef_ on classifier. Cannot plot feature importances.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
C:\Users\thu71\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
wandb: Logged summary metrics.
wandb: Logged class proportions.
C:\Users\thu71\AppData\Roaming\Python\Python39\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\thu71\AppData\Roaming\Python\Python39\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: Conve

accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.69369
auc,0.73071
b_accuracy,0.5
f1,0.0
